# **📊 DSCBI Macroeconomic Indicators Visualization Project**

## **Description:**
* The DSCBI Visualization Project aims to create an interactive and dynamic dashboard using Streamlit and Python to visualize macroeconomic indicators produced by the department.
* It will provide stakeholders with quick insights into sectoral performance, indicator trends, and projections for planning and policy analysis.
## **🎯 Objectives**
* Centralize visualization of macroeconomic indicators (historical and projected).
* Enable users to filter and view data by sector, indicator, and year range.
* Provide key metric summaries (average, max, and long-term projected values).
* Present interactive charts for both calendar year and fiscal year data.
* Offer a downloadable table view for selected indicators.
* Highlight data sources, stakeholders, and important reference links.

## **🧱 Project Structure**


dscbi_visualization/
│
├── data/
│   ├── macro_indicators.csv
│   └── metadata_sources.csv
│
├── app/
│   ├── __init__.py
│   ├── main.py                # Main Streamlit entry point
│   ├── utils.py               # Helper functions for loading data, computing metrics
│   ├── charts.py              # Chart generation functions
│   ├── layout.py              # Layout and style configurations
│
├── assets/
│   ├── logo.png
│   └── style.css
│
├── requirements.txt
├── README.md
└── .gitignore


## **🧭 Dashboard Layout**

| Section                                   | Description                                                                                  |
| ----------------------------------------- | -------------------------------------------------------------------------------------------- |
| **1. Sidebar (Selection Panel)**          | Contains filters for: <br>– Sector Selection <br>– Indicator Selection <br>– Years Selection |
| **2. Title Header**                       | Attractive title banner with department logo, project title, and tagline.                    |
| **3. Main Body**                          | Divided into three key parts:                                                                |
|     • **Key Metrics Panel**               | Displays summary indicators (Max value, Average value, Long-term projection).                |
|     • **Charts Section**                  | Two charts: <br>– Calendar Year Trends <br>– Fiscal Year Trends                              |
|     • **Outputs Table**                   | Interactive table displaying data based on selected filters (with historical + projections). |
| **4. Key Sources & Stakeholders Section** | Displays key institutions, data providers, and important links for reference.                |


### User Interaction

* User selects Sector, Indicator(s), and Year Range from the sidebar.

### Data Filtering

* The app dynamically filters the dataset based on selected parameters.

### Computation

* Metrics (Average, Max, Long-term projection) are calculated.

### Visualization

#### The filtered data is plotted into two main charts:

* Calendar Year Chart

* Fiscal Year Chart

### Table Generation

* A dynamic table displays the corresponding data.

### Reference Section

* Displays key sources and links at the bottom.

In [21]:
# ==============================
# 📦 SECTION 1: IMPORT LIBRARIES
# Load required Python libraries for Streamlit app, data handling, and visualization.
# ==============================
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from io import StringIO
import re
from pathlib import Path

# display columns and rows settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# ==============================




In [ ]:
# ==============================
# 🧭 SECTION 2: PAGE CONFIGURATION
# Configure Streamlit page layout, title, favicon, and custom CSS styles.
# ==============================
st.set_page_config(
    page_title="DSCBI Macroeconomic Indicators Visualization Project",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)
st.markdown("""
    <style>
        .reportview-container {
            background: #f0f2f5;
        }
        .sidebar .sidebar-content {
            background: #ffffff;
        }
    </style>
""", unsafe_allow_html=True)
# ==============================
# ==============================


In [25]:
DIR_DATA =  Path.cwd().parents[1].joinpath("DSCBI_PROJECT", "data")
FILE = DIR_DATA.joinpath("Basic_Macro_Indicators.csv")

In [38]:
df = pd.read_csv(FILE)
df.head()

,Sector,Indicator,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025 Proj,2026 Proj,2027 Proj,2028 Proj,2029 Proj,2030 Proj
0,Real Sector,Population growth,10.2,10.5,10.7,11,11.3,11.5,11.8,12.1,12.4,12.7,13,13.2,13.5,13.8,14.1,14.4,14.7,15.1,15.4,15.7
1,Real Sector,Annual change in Population (%),2.00%,2.90%,1.90%,2.80%,2.70%,1.80%,2.40%,2.50%,2.50%,2.40%,2.40%,1.60%,2.20%,2.20%,2.20%,2.20%,2.20%,2.20%,2.20%,2.20%
2,Real Sector,Nominal GDP (in billion Frw) (FY),"3,778","4,373","4,846","5,296","5,771","6,443","7,237","7,982","8,784","9,640","10,499","12,359","15,800","18,371","21,512","24,413","27,692","31,146","34,914","39,155"
3,Real Sector,Nominal GDP (in billion Frw) (CY),"4,084","4,642","4,988","5,548","6,081","6,768","7,621","8,310","9,461","9,893","11,125","14,164","17,176","19,918","22,817","26,010","29,374","32,919","36,909","41,400"
4,Real Sector,Nominal GDP (million $) (CY),"6,803","7,557","7,714","8,130","8,451","8,597","9,166","9,651","10,498","10,467","11,233","13,712","14,704","15,058","16,080","17,499","19,014","20,653","22,562","24,690"


In [43]:
df["Sector"].unique()

array(['Real Sector', 'External sector', 'Debt', 'Fiscal sector',
       'Monetary sector'], dtype=object)

In [ ]:
# ==============================
# 🗂️ SECTION 3: LOAD AND CACHE DATA
# Load macroeconomic indicator data and metadata.
# Use st.cache_data to improve performance and prevent reloading each time.
# ==============================
@st.cache_data
def load_data(file):
    data = pd.read_csv(file)
    data = data[0:66]
    #drop all unnamed columns
    data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
    #melting the dataframe
    data = data.melt(id_vars=['Sector', 'Indicator'], var_name='Year', value_name='Value')

    # search the Value column in df_melted if whether the value contains a comma and remove it and also a percent sign if any
    data['Value'] = data['Value'].apply(lambda x: re.sub(r'[,%]', '', str(x)) if isinstance(x, str) else x)

    data['Year'] = data['Year'].apply(lambda x: re.sub(r'[ Proj]', '', str(x)) if isinstance(x, str) else x)

    data['Value'] = pd.to_numeric(data['Value'], errors='coerce')
    return data
data = load_data(FILE)






In [15]:
data.columns

Index(['Sector', 'Indicator', 'Year', 'Value'], dtype='object')

In [ ]:
list(data.Indicator.unique())

In [ ]:
# ==============================
# 🎛️ SECTION 4: SIDEBAR SELECTION PANEL
# Sidebar filters for sector, indicator, and year selection.
# This drives all downstream data filtering and visualizations.
# ==============================
st.sidebar.header("Filter Options")
sector_options = ['Real Sector', 'External sector', 'Debt', 'Fiscal sector', 'Monetary sector']
selected_sector = st.sidebar.selectbox("Select Sector", sector_options)
indicator_options = list(data.Indicator.unique())
selected_indicator = st.sidebar.selectbox("Select Indicator", indicator_options)
year_options = list(data.Year.unique())
selected_year = st.sidebar.selectbox("Select Year", year_options)
# ==============================
# ==============================


In [ ]:
# ==============================
# 🏷️ SECTION 5: DASHBOARD TITLE HEADER
# Create an appealing header with project title, logo, and description text.
# ==============================
st.title("📊 DSCBI Macroeconomic Indicators Visualization Project")
st.markdown("""
    <style>
        .title {
            font-size: 2em;
            font-weight: bold;
            color: #333;
        }
    </style>
""", unsafe_allow_html=True)
st.image("https://example.com/logo.png", width=100)
st.markdown("""
    <div class="DSCBI Macroeconomic Indicators Visualization Project</div>
    <p>The DSCBI Visualization Project aims to create an interactive and dynamic dashboard using Streamlit and Python to visualize macroeconomic indicators produced by the department.</p>
""", unsafe_allow_html=True)
# ==============================
# ==============================

In [ ]:
# ==============================
# 📈 SECTION 6: KEY METRICS DISPLAY
# Compute and show summary metrics (max, average, min) for selected indicators from above filter options.
# ==============================




In [ ]:
# ==============================
# 📊 SECTION 7: CHARTS SECTION
# Generate two main plots:
#  - Calendar Year Indicator Chart
#  - Fiscal Year Indicator Chart
# Use Plotly or Altair for interactivity.
# ==============================
def plot_indicator_chart(data, indicator, year_type):
    if indicator == 'All':
        return
    if year_type == 'Calendar':
        fig = px.line(data, x='Month', y=indicator, title=f'Calendar Year {indicator} Chart')
    else:
        fig = px.line(data, x='Fiscal Month', y=indicator, title=f'Fiscal Year {indicator} Chart')
    st.plotly_chart(fig)
data = load_data('customer_churn_data.csv')
plot_indicator_chart(data, selected_indicator, 'Calendar')
plot_indicator_chart(data, selected_indicator, 'Fiscal')
# ==============================
# ==============================

In [ ]:
# ==============================
# 📋 SECTION 8: OUTPUT TABLE
# Display a filtered data table showing historical and projected values for the selected indicators.
# Include data download option (CSV/Excel).
# ==============================
def display_data_table(data, indicator):
    if indicator == 'All':
        st.dataframe(data)
    else:
        filtered_data = data[['Month', indicator]]
        st.dataframe(filtered_data)
        csv = filtered_data.to_csv(index=False).encode('utf-8')
        st.download_button(
            label="Download data as CSV",
            data=csv,
            file_name='customer_churn_data.csv',
            mime='text/csv',
        )
data = load_data('customer_churn_data.csv')
display_data_table(data, selected_indicator)
# ==============================
# ==============================

In [ ]:
# ==============================
# 🌐 SECTION 9: SOURCES & LINKS
# Display information on data sources, key stakeholders, and related web links.
# ==============================
st.sidebar.header("Data Sources & Links")
st.sidebar.markdown("""
- [Customer Churn Data](https://example.com/customer_churn_data)
- [Key Stakeholders](https://example.com/key_stakeholders)
- [Related Research](https://example.com/related_research)
""")
# ==============================
# ==============================

In [ ]:
# ==============================
# 🚀 SECTION 10: RUN THE APP
# Streamlit entry point for running the dashboard application.
# ==============================
if __name__ == "__main__":
    st.write("Customer Churn Analysis Dashboard is running...")
# ==============================
# ==============================